In [ ]:
import numpy as np

from google.colab import drive
import tensorflow as tf

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('sample_data/island.txt', 'r') as f:
    text = f.read()

In [ ]:
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
text = text[start_indx:end_indx]
char_set = set(text)

print('Total Length:', len(text))

Total Length: 1130710


In [ ]:
print('Unique Characters:', len(char_set))

Unique Characters: 85


In [ ]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i, ch in enumerate(chars_sorted) }
char_array = np.array(chars_sorted)

In [ ]:
text_encoded = np.array([char2int[ch] for ch in text], dtype=np.int32)
print("Text encoded shape: ", text_encoded.shape)

Text encoded shape:  (1130710,)


In [ ]:
print(text[:15], '=== Encoded ==> ', text_encoded[:15])

THE MYSTERIOUS  === Encoded ==>  [48 36 33  1 41 53 47 48 33 46 37 43 49 47  1]


In [ ]:
print(text_encoded[15:35], '=== reverse ==>', ''.join(char_array[text_encoded[15:35]]))

[37 47 40 29 42 32  1 10 10 10  0  0  0  0  0 48 36 33  1 41] === reverse ==> ISLAND ***




THE M


In [ ]:
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for example in ds_text_encoded.take(5):
    print('{} --> {}'.format(example.numpy(), char_array[example.numpy()]))

48 --> T
36 --> H
33 --> E
1 -->  
41 --> M


In [ ]:
seq_len = 50
chunk_size = seq_len + 1

ds_chunks = ds_text_encoded.batch(batch_size=chunk_size, drop_remainder=True)

In [ ]:
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]

    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

In [ ]:
for sample in ds_sequences.take(2):
    print('  Input (x) : ', repr(''.join(char_array[sample[0].numpy()])))
    print('  Target (y)  : ', repr(''.join(char_array[sample[1].numpy()])))
    print()

  Input (x) :  'THE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTERIOUS ISLAN'
  Target (y)  :  'HE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTERIOUS ISLAND'

  Input (x) :  '\n\nby Jules Verne\n\n1874\n\n\n\n\nPART 1--DROPPED FROM TH'
  Target (y)  :  '\nby Jules Verne\n\n1874\n\n\n\n\nPART 1--DROPPED FROM THE'



In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

ds = ds_sequences.shuffle(BUFFER_SIZE).batch(batch_size=BATCH_SIZE)

#### Building a character-level RNN model.

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])

    return model

In [ ]:
#Set training parameters.
charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

In [ ]:
tf.random.set_seed(42)

model = build_model(vocab_size=charset_size,
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         21760     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense (Dense)               (None, None, 85)          43605     
                                                                 
Total params: 1640277 (6.26 MB)
Trainable params: 1640277 (6.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(ds, epochs=20)

Epoch 1/20
347/347 [==============================] - 13s 20ms/step - loss: 2.4193
Epoch 2/20
347/347 [==============================] - 7s 17ms/step - loss: 1.8209
Epoch 3/20
347/347 [==============================] - 6s 15ms/step - loss: 1.6066
Epoch 4/20
347/347 [==============================] - 7s 16ms/step - loss: 1.4830
Epoch 5/20
347/347 [==============================] - 6s 15ms/step - loss: 1.4002
Epoch 6/20
347/347 [==============================] - 8s 19ms/step - loss: 1.3422
Epoch 7/20
347/347 [==============================] - 6s 15ms/step - loss: 1.2981
Epoch 8/20
347/347 [==============================] - 6s 15ms/step - loss: 1.2623
Epoch 9/20
347/347 [==============================] - 7s 16ms/step - loss: 1.2328
Epoch 10/20
347/347 [==============================] - 6s 16ms/step - loss: 1.2078
Epoch 11/20
347/347 [==============================] - 7s 15ms/step - loss: 1.1867
Epoch 12/20
347/347 [==============================] - 8s 20ms/step - loss: 1.1668
Epoch 13/20


#### Evaluation phase – generating new text passages.

In [ ]:
tf.random.set_seed(42)

logits = [[1.0, 1.0, 1.0]]
print('Probabilities: ', tf.math.softmax(logits).numpy()[0])

Probabilities:  [0.33333334 0.33333334 0.33333334]


In [ ]:
samples = tf.random.categorical(logits=logits, num_samples=10)
tf.print(samples.numpy())

array([[0, 2, 1, 2, 2, 0, 2, 2, 0, 1]])


In [ ]:
def sample(model, starting_str,
           len_generated_txt=700,
           max_input_length=50, scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str
    model.reset_states()
    for i in range(len_generated_txt):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_idx = tf.random.categorical(scaled_logits, num_samples=1)
        new_char_idx = tf.squeeze(new_char_idx)[-1].numpy()

        generated_str += str(char_array[new_char_idx])
        new_char_idx = tf.expand_dims([new_char_idx], 0)
        encoded_input = tf.concat([encoded_input, new_char_idx], axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

In [ ]:
tf.random.set_seed(1)
print(sample(model, starting_str='Machine Learning'))

Machine Learning, at its four words,”
said the reporter.

“Telliged the base were regarly to work where you have been almost unable to
least also four enormous support of the coast. He wished to return to the box wounded.

“Hullocks now even comes in the depths; to the pressure was to be no manquived with the plateau of Prospect Heights! The ore of the sky
was certoring in to fear it opportant provine some time, and in proad and such as on the periments,
its sun statissude of fiercess, that Neb returned, and that
even if he was difinedred! What do it was then he had lowered
above the last of the Mercy, succumbless in the wood.

“My med!” cried Neb, Ayrton, and Pondisting, transformed into the sea.

Moreover


In [ ]:
model.save('Island.keras')

In [ ]:
loaded = tf.keras.models.load_model('Island.keras')

In [ ]:
print(sample(loaded, starting_str='This life is not', len_generated_txt=5000))

This life is not to- the island came one of Poly so as to
arrive her to staped and who brisking his sailor beasts.

The storm rowed upon every one of that east more ony for
others shout the colonists to gain. In exploration, rose happened with a little so that the brig, whose showest mineral forests of the bullet. It was always
parts of the island, Spiceats ascended to the southwest.

But it was to attack for a long sudden ground, thanks of the
productions of their dwates which forms, which might indicaries, and previsious occurred in the
body, and the reporter’s gate, it is formerly clearing in the
world. These watered at two miles. The next day was this. He then could veowed that water, but they did not still a northern parts of the Mercy, standing during the shore.

Twister forces to the volcanic aculor, and beat at the islet on the other from the
skints. Foredar placember, as well wires that in at what
willand fevel of you by that part of this limisting arrived at
a massacripilator

In [ ]:
print(sample(loaded, starting_str='Ngwendete muno', len_generated_txt=500, scale_factor=2.0))

Ngwendete munor of the balloon, and to the sea for either the convicts, who were still living!”

And the reporter was ready to do this incident, and the colonists appeared as if the colonists were not only of the plateau of Prospect Heights.

The sailor said to the southern part of the lake, and
the stars of the convicts were the last time to the opening
both of the islet when the sailor and the engineer only been so fortunately, and at a sort of all side it would be towards the storms which had stopped by th


In [ ]:
print(sample(loaded, starting_str='Ngwendete muno', len_generated_txt=500, scale_factor=0.5))

Ngwendete munord rloid to hime, noo Amincian” has creater, to, Pearchiff
and
Capon, set throight highsrittsy a kirgp. 
His nime report you!” shouted. The blvewelt duellwar
anvanytha. Noh!” replied Ayrton By Cluswer
to oag. It is!” said Punafasjaps. Tsleinds
cropced, thorough tim--”
3, gailitaryhwescepts. As sayblandous.” “Byried the squout lukina zenitionlage.o.;.. Ef follow Richtroascenine northwival you creepers to term ylass!”

Furing its watco the “was, cart!
tracs to  fip, Heaply.

“Bo onifie to receiet 


In [ ]:
print(sample(loaded, starting_str='Ngwendete muno', len_generated_txt=500, scale_factor=5.0))

Ngwendete munor of the convicts, and the convicts were all the stream which would be allowed the convicts, and the settlers were in the bottom of the islet, and the sailor and the sailor and the sailor returned to the southern spurs of the coast.

The sailor and the sailor and the sailor and his companions were still the sun without stopping the storm of the islet, and the convicts were all the shore of the islet, and the convicts would have been able to struggle a little band, the settlers were the convicts 


In [ ]:
print(sample(loaded, starting_str='The island', len_generated_txt=1000, scale_factor=1.5))

The island appear and replication or any minute is not only this case would have been liked that the brig approached the thirty-engineer. The
injurced was the creek on this storms which had appeared to bring in the far from the northeast, and the object of a battery of the island
in the world in the shore. It would be our trees mars of the submersions of the shore. The ape of a continent of the stones sprang up. The light of the atmosphere alone was to bring the passage of the
interior of the lake, so as to try the sources of the coast of the cone was ended with the prisoners of the brig became returned to the southern parts of the lake, and yet they would be transformed into the photoet
of the islet on the convicts, and the most discovery men away all the stream, when the settlers had been searched the gentle stable. The hind of
the brig would be set forth in consequence.

“You are right, he contented the least still more very fresh parts of the United States, or the heap of a busines